# Reporting summary:
- Dependent variables
 - Positive return days > Negative return days
 - The skewness being much lower
 - Being much more interpretable
- Independent variables
 - No missing values
 - The skewness being much lower
 - The meaning of log return variables interpretable as the change of recent patterns
- ADF test: All used variables being stationary
- Ljung & Box test: Dependent variable not being autocorrelated


# Import libraries and the data

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 999)
pd.options.display.float_format = '{:.4f}'.format
from tabulate import tabulate
from matplotlib import pyplot as plt
import seaborn as sns
import ast
import itertools
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import glob as glob
from scipy.stats import norm
from scipy.stats import probplot
import statsmodels.api as sm
from scipy.stats import t as t_dist
from scipy.stats import describe
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.simplefilter('ignore')

path = "..\data"

# Import the user-defined modules
import sys
sys.path.append(path+'\..\src')
import preprocess_ts, conduct_eda, evaluate_results
from preprocess_ts import *
from conduct_eda import *
from evaluate_results import *

In [2]:
btc = pd.read_pickle(path+"/02_intermediate/dv.pickle")[: "2021-02-21"]
blockchain = pd.read_pickle(path+"/02_intermediate/blockchain.pickle")[: "2021-02-21"]
macro = pd.read_pickle(path+"/02_intermediate/macro.pickle")[: "2021-02-21"]
crypto = pd.read_pickle(path+"/02_intermediate/crypto.pickle")[: "2021-02-21"]
df = pd.read_pickle(path+"/03_primary/blockchain_altcoin_macro.pickle")[: "2021-02-21"]
dv = df[["BTC_Price", "BTC_Price_LogReturn", "BTC_Price_LogReturn_binary"]][: "2021-02-21"]
iv = df.drop(["BTC_Price", "BTC_Price_LogReturn_binary"], axis=1)[: "2021-02-21"]

# Focus on important variables chosen by Boruta
important_features = output_boruta()

# Definition of variables

In [3]:
price_volume = list(macro.columns) + list(crypto.columns)
original_price_volume= [p_v for p_v in price_volume if ("_log" not in p_v) & ("_LogReturn" not in p_v)]

vars_list = list(btc.columns) + list(blockchain.columns)
original_vars = [var for var in vars_list if ("_log" not in var) & ("_LogReturn" not in var)]

dict_definition_vars = {
    'btc_price': "The average USD market price across major bitcoin exchanges.(USD)",
    'total-bitcoins': "The total number of mined bitcoin that are currently circulating on the network.(BTC)",
    'market-cap': "The total USD value of bitcoin in circulation.(USD)",
    'trade-volume': "The total USD value of trading volume on major bitcoin exchanges.(USD)",
    'blocks-size': "The total size of the blockchain minus database indexes in megabytes.(MB)",
    'avg-block-size': "The average block size over the past 24 hours in megabytes.(MB)",
    'n-transactions-per-block': "The average number of transactions per block over the past 24 hours.(transaction)",
    'n-transactions-total': "The total number of transactions on the blockchain.(transaction)",
    'median-confirmation-time': "The median time for a transaction with miner fees to be included in a mined block and added to the public ledger.(min)",
    'avg-confirmation-time': "The average time for a transaction with miner fees to be included in a mined block and added to the public ledger.(min)",
    'hash-rate': "The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.(TH/s)",
    'difficulty': "A relative measure of how difficult it is to mine a new block for the blockchain.(relative)",
    'miners-revenue': "Total value of coinbase block rewards and transaction fees paid to miners.(USD)",
    'transaction-fees': "The total BTC value of all transaction fees paid to miners.(BTC)",
    'cost-per-transaction': "Miners revenue divided by the number of transactions.(USD)",
    'n-unique-addresses': "The total number of unique addresses used on the blockchain.(address)",
    'n-transactions': "The total number of confirmed transactions per day.(transaction)",
    'transactions-per-second': "The number of transactions added to the mempool per second.(transaction/s)",
    'output-volume': "The total value of all transaction outputs per day.(BTC)",
    'mempool-count': "The total number of unconfirmed transactions in the mempool.(transaction)",
    'mempool-growth': "The rate at which the mempool is growing in bytes per second.(byte/s)",
    'mempool-size': "The aggregate size in bytes of transactions waiting to be confirmed.(byte)",
    'utxo-count': "The total number of valid unspent transactions outputs.(transaction)",
    'n-transactions-excluding-popular': "The total number of transactions excluding those involving the network's 100 most popular addresses.(transaction)",
    'estimated-transaction-volume': "The total estimated value in BTC of transactions on the blockchain.(BTC)",
    'mvrv': "Market Value divided by Realised Value.(ratio)",
    'nvts': "The Moving Average over the last 90 days of the denominator of Network Value (= Market Value) by the total transactions volume in USD over the past 24hour.(ratio)",
}

df_definition_vars = pd.DataFrame([])
df_definition_vars["Variables"] = dict_definition_vars.keys()
df_definition_vars["Types of Variables"] = "BTC blockchain"
df_definition_vars["Definition of Variable"] = dict_definition_vars.values()

display(df_definition_vars)
df_definition_vars.to_csv(path+"/08_reporting/tableA1_definition_variables.csv")

,Variables,Types of Variables,Definition of Variable
0,btc_price,BTC blockchain,The average USD market price across major bitc...
1,total-bitcoins,BTC blockchain,The total number of mined bitcoin that are cur...
2,market-cap,BTC blockchain,The total USD value of bitcoin in circulation....
3,trade-volume,BTC blockchain,The total USD value of trading volume on major...
4,blocks-size,BTC blockchain,The total size of the blockchain minus databas...
5,avg-block-size,BTC blockchain,The average block size over the past 24 hours ...
6,n-transactions-per-block,BTC blockchain,The average number of transactions per block o...
7,n-transactions-total,BTC blockchain,The total number of transactions on the blockc...
8,median-confirmation-time,BTC blockchain,The median time for a transaction with miner f...
9,avg-confirmation-time,BTC blockchain,The average time for a transaction with miner ...


# Descriptive Statistics

## Dependent Variable

In [4]:
check_data(dv)

(1263, 3)
            BTC_Price  BTC_Price_LogReturn  BTC_Price_LogReturn_binary
Date                                                                  
2017-09-08  4613.8400               0.0004                      1.0000
2017-09-09  4312.2600              -0.0676                      0.0000
2017-09-10  4330.8700               0.0043                      1.0000
            BTC_Price  BTC_Price_LogReturn  BTC_Price_LogReturn_binary
Date                                                                  
2021-02-19 51608.1500              -0.0098                      0.0000
2021-02-20 55916.5000               0.0802                      1.0000
2021-02-21 56001.2000               0.0015                      1.0000
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1263 entries, 2017-09-08 to 2021-02-21
Freq: D
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BTC_Price                   1

In [5]:
desc = describe_summary_statistics(dv[["BTC_Price_LogReturn_binary"]]).T
desc["count"] = desc["count"].astype("int")
desc = desc[["count", "mean", "std", "min", "50%", "max", "skewness"]]
display(desc)
desc.to_csv(path+"/08_reporting/tableData_descriptive_statistics_preprocessed_DVs.csv")

,count,mean,std,min,50%,max,skewness
BTC_Price_LogReturn_binary,1263,0.5376,0.4988,0.0000,1.0000,1.0000,-0.1509


### Visualization

Bitcoin prices

In [6]:
fig = plt.figure(figsize=(10,8), dpi=800)
ax1 = plt.subplot2grid((2,2), (0,0))
ax2 = plt.subplot2grid((2,2), (0,1))
ax3 = plt.subplot2grid((2,2), (1,0), colspan=2)

ax1.boxplot(df["BTC_Price"])
ax1.set_title("Boxplot: Bitcoin prices")
ax1.set_ylabel("Prices (USD)")
ax1.grid(True)

ax2.hist(df["BTC_Price"], bins=100)
ax2.set_title("Histogram: Bitcoin prices")
ax2.set_xlabel("Prices (USD)")
ax2.set_ylabel("Frequency (Daily)")
ax2.grid(True)

ax3.plot(df["BTC_Price"])
ax3.set_title("Time Series: Bitcoin prices")
ax3.set_xlabel("Date (Daily)")
ax3.set_ylabel("Prices (USD)")
ax3.grid(True)

fig.tight_layout()
fig.show()
plt.savefig(path+"/08_reporting/figureData_BTC_prices.png", format="png", dpi=800)

Log Return of Bitcoin prices

In [7]:
fig = plt.figure(figsize=(10,8), dpi=800)
ax1 = plt.subplot2grid((2,2), (0,0))
ax2 = plt.subplot2grid((2,2), (0,1))
ax3 = plt.subplot2grid((2,2), (1,0), colspan=2)

ax1.boxplot(df["BTC_Price_LogReturn"])
ax1.set_title("Boxplot: Log return of Bitcoin prices")
ax1.set_ylabel("Log return of Bitcoin prices (Unit ratio)")
ax1.grid(True)

ax2.hist(df["BTC_Price_LogReturn"], bins=100)
ax2.set_title("Histgram: Log return of Bitcoin prices")
ax2.set_xlabel("Log return of Bitcoin prices (Unit ratio)")
ax2.set_ylabel("Frequency (Daily)")
ax2.grid(True)

ax3.plot(df["BTC_Price_LogReturn"])
ax3.set_title("Time Series: Log return of Bitcoin prices")
ax3.set_xlabel("Date (Daily)")
ax3.set_ylabel("Log return of Bitcoin prices (Unit ratio)")
ax3.grid(True)

fig.tight_layout()
fig.show()
plt.savefig(path+"/08_reporting/figureData_BTC_LogReturn.png", format="png", dpi=800)

## Independent variables

In [8]:
check_data(iv)

(1263, 79)
            BTC_Price_LogReturn  market-cap_LogReturn  trade-volume_LogReturn  \
Date                                                                            
2017-09-08               0.0004               -0.0117                 -0.5434   
2017-09-09              -0.0676               -0.0279                  0.8755   
2017-09-10               0.0043               -0.0301                 -0.7541   

            avg-block-size_LogReturn  n-transactions-per-block_LogReturn  \
Date                                                                       
2017-09-08                    0.0554                              0.0920   
2017-09-09                   -0.0996                             -0.2139   
2017-09-10                    0.0298                              0.0825   

            median-confirmation-time_LogReturn  \
Date                                             
2017-09-08                             -0.3159   
2017-09-09                             -0.1274   
20

In [9]:
important_features

['BCH_Price',
 'BCH_Volume',
 'BNB_Price',
 'BTC_Price',
 'EOS_Price',
 'ETC_Price',
 'ETH_Price',
 'IOTA_Price',
 'IOTA_Volume',
 'LTC_Price',
 'NEO_Price',
 'USDT_Price',
 'USDT_Volume',
 'WAVES_Price',
 'XEM_Price',
 'XLM_Price',
 'XMR_Price',
 'XRP_Price',
 'XRP_Volume',
 'ZEC_Price',
 'cost-per-transaction',
 'gbp_usd_Price',
 'gold_Price',
 'market-cap',
 'mempool-count',
 'mempool-size',
 'mvrv',
 'nvts',
 'utxo-count']

In [10]:
# Extract the important variables
important_features_LogReturn = output_boruta(LogReturn=True)
desc = describe_summary_statistics(iv)[important_features_LogReturn].T
desc["count"] = desc["count"].astype("int")
desc = desc.drop(["25%", "75%", "kurtosis"],axis=1)
display(desc)
desc.to_csv(path+"/08_reporting/tableData_descriptive_statistics_preprocessed_IVs.csv")

,count,mean,std,min,50%,max,skewness
BCH_Price_LogReturn,1263,0.0002,0.0687,-0.5585,-0.0018,0.4310,0.0096
BCH_Volume_LogReturn,1263,0.0015,0.3233,-1.2390,-0.0222,2.1469,0.8022
BNB_Price_LogReturn,1263,0.0040,0.0577,-0.5027,0.0009,0.5271,0.7713
BTC_Price_LogReturn,1263,0.0020,0.0436,-0.4965,0.0023,0.1971,-1.1150
EOS_Price_LogReturn,1263,0.0010,0.0534,-0.3545,0.0000,0.3043,0.2188
ETC_Price_LogReturn,1263,0.0000,0.0554,-0.4230,0.0005,0.3004,-0.8237
ETH_Price_LogReturn,1263,0.0015,0.0522,-0.5470,0.0007,0.2343,-1.1067
IOTA_Price_LogReturn,1263,0.0004,0.0310,-0.3043,-0.0001,0.2673,0.4749
IOTA_Volume_LogReturn,1263,0.0009,0.3879,-1.7307,-0.0234,1.8255,0.4488
LTC_Price_LogReturn,1263,0.0009,0.0563,-0.4377,0.0000,0.3878,0.0956


# Time Series Analysis

## Stationarity Testing (ADF testing)



### Dependent Variable

In [11]:
test_adf(dv)

,test statistic,p_value
variables,,
BTC_Price,4.3455,1.0000
BTC_Price_LogReturn,-24.5180,0.0000
BTC_Price_LogReturn_binary,-38.3548,0.0000


### Independent Variables

In [12]:
test_adf(iv[important_features_LogReturn]).to_csv(path+"/08_reporting/tableData_adf_IVs_preprocessed.csv")

## Ljung & Box test

### Dependent Variable

We could not find statistically significant autocorrelation in Log return of Bitcoin prices over the weeks.

In [13]:
sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[7])

,lb_stat,lb_pvalue
7,13.5166,0.0605


In [14]:
sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[14])

,lb_stat,lb_pvalue
14,20.3328,0.1200


In [15]:
print(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[20]))
print(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[40]))
print(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[60]))

    lb_stat  lb_pvalue
20  30.9171     0.0563
    lb_stat  lb_pvalue
40  46.4199     0.2247
    lb_stat  lb_pvalue
60  66.2314     0.2707


In [16]:
round_digits3 = lambda float_value: round(float(float_value), ndigits=3)
pd.DataFrame({"lags": [20,40,60], 
              "test statistic": [round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[20])[0]),
                                 round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[40])[0]),
                                 round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[60])[0])], 
              "p_value": [round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[20])[1]),
                          round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[40])[1]),
                          round_digits3(sm.stats.acorr_ljungbox(dv["BTC_Price_LogReturn"], lags=[60])[1])
                          ]}).to_csv(path+"/08_reporting/tableData_ljung_box.csv", index=False)

KeyError: 0

# Show the mastertable

In [17]:
df.drop("BTC_Price", axis=1)

,BTC_Price_LogReturn_binary,BTC_Price_LogReturn,market-cap_LogReturn,trade-volume_LogReturn,avg-block-size_LogReturn,n-transactions-per-block_LogReturn,median-confirmation-time_LogReturn,avg-confirmation-time_LogReturn,hash-rate_LogReturn,difficulty_LogReturn,...,BNB_Price_LogReturn,XLM_Price_LogReturn,EOS_Price_LogReturn,XMR_Price_LogReturn,XEM_Price_LogReturn,NEO_Price_LogReturn,IOTA_Price_LogReturn,ZEC_Price_LogReturn,ETC_Price_LogReturn,WAVES_Price_LogReturn
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-08,1.0000,0.0004,-0.0117,-0.5434,0.0554,0.0920,-0.3159,0.5273,-0.0057,0.0000,...,-0.0088,-0.0003,-0.0227,-0.0150,-0.0011,-0.1191,0.0131,-0.0009,-0.0192,-0.0088
2017-09-09,0.0000,-0.0676,-0.0279,0.8755,-0.0996,-0.2139,-0.1274,-0.7249,-0.0349,0.0000,...,-0.0240,-0.0008,-0.0294,-0.0342,-0.0082,0.0338,-0.0427,-0.0041,-0.0805,-0.0087
2017-09-10,1.0000,0.0043,-0.0301,-0.7541,0.0298,0.0825,0.5293,0.5447,-0.2026,0.0000,...,0.0422,0.0007,-0.0051,0.0037,-0.0039,-0.0374,0.0410,-0.0195,0.0624,-0.0116
2017-09-11,0.0000,-0.0229,-0.0191,0.2240,0.0250,0.0396,-0.4683,-0.3327,0.2318,0.0000,...,0.0049,-0.0007,0.0590,-0.0076,-0.0024,-0.0403,0.0214,-0.0048,-0.0106,0.0060
2017-09-12,0.0000,-0.0081,0.0238,-0.1818,-0.0667,-0.0666,0.1279,-0.3971,0.0774,0.0000,...,-0.0496,-0.0010,-0.0601,-0.0050,-0.0123,-0.0424,-0.0223,-0.0518,-0.0216,-0.0835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-17,1.0000,0.0249,0.0348,0.2385,-0.0025,-0.0621,-0.0177,0.2167,0.1315,0.0000,...,0.2344,0.0070,0.0336,0.2018,0.0116,0.0365,0.0182,0.0824,0.0566,0.0304
2021-02-18,1.0000,0.0584,0.0201,0.1753,0.0061,0.0050,0.0284,0.2712,-0.0073,0.0000,...,0.1729,0.0003,-0.0006,-0.0329,0.0280,0.0333,0.0744,-0.0022,-0.0173,0.0752
2021-02-19,0.0000,-0.0098,0.0181,-0.4765,0.0102,-0.0356,-0.0409,0.0331,0.1114,0.0010,...,0.5271,0.0107,0.0667,0.0778,0.0595,0.0833,0.0041,0.0686,0.0251,0.0145
